In [1]:
!sudo apt-get install -y fonts-nanum*
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'fonts-nanum-extra' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-coding' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-eco' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum' for glob 'fonts-nanum*'
The following NEW packages will be installed:
  fonts-nanum fonts-nanum-coding fonts-nanum-eco fonts-nanum-extra
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 46.0 MB of archives.
After this operation, 177 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum-coding all 2.5-3 [4,988 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum-eco all 1.000-7 [14.7 MB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum-extra all 20200506-1 [21.0

In [2]:
!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

In [ ]:
# https://github.com/wikibook/pytorchdl2/blob/main/notebooks/ch05_regression.ipynb

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# 폰트 관련 용도
import matplotlib.font_manager as fm


import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot

plt.rc('font', family='NanumBarunGothic')

#### 1입력 1출력

In [3]:
torch.manual_seed(123)

l1 = nn.Linear(1, 1)

print(l1)

Linear(in_features=1, out_features=1, bias=True)


In [4]:
for param in l1.named_parameters():
    print('name: ', param[0])
    print('tensor: ', param[1])
    print('shape: ', param[1].shape)


name:  weight
tensor:  Parameter containing:
tensor([[-0.4078]], requires_grad=True)
shape:  torch.Size([1, 1])
name:  bias
tensor:  Parameter containing:
tensor([0.0331], requires_grad=True)
shape:  torch.Size([1])


In [5]:
#명시적 초기화
nn.init.constant_(l1.weight, 2.0)
nn.init.constant_(l1.bias, 1.0)

print(l1.weight)
print(l1.bias)

Parameter containing:
tensor([[2.]], requires_grad=True)
Parameter containing:
tensor([1.], requires_grad=True)


In [6]:
x_np = np.arange(-2,2.1,1)
x = torch.tensor(x_np, dtype = torch.float32)
x = x.view(-1,1)
x.shape, x

(torch.Size([5, 1]),
 tensor([[-2.],
         [-1.],
         [ 0.],
         [ 1.],
         [ 2.]]))

In [8]:
y = l1(x)
y.shape, y.data

(torch.Size([5, 1]),
 tensor([[-3.],
         [-1.],
         [ 1.],
         [ 3.],
         [ 5.]]))

2입력 1출력

In [9]:
l2 = nn.Linear(2, 1)
nn.init.constant_(l2.weight, 1.)
nn.init.constant_(l2.bias, 2.)
l2.weight, l2.bias

(Parameter containing:
 tensor([[1., 1.]], requires_grad=True),
 Parameter containing:
 tensor([2.], requires_grad=True))

In [12]:
x2 = torch.tensor([[0, 0],
                   [0, 1],
                   [1, 0],
                   [1, 1]], dtype = torch.float32)

x2.shape, x2

(torch.Size([4, 2]),
 tensor([[0., 0.],
         [0., 1.],
         [1., 0.],
         [1., 1.]]))

In [13]:
y2 = l2(x2)
y2.shape, y2.data

(torch.Size([4, 1]),
 tensor([[2.],
         [3.],
         [3.],
         [4.]]))

2입력 3출력

In [27]:
l3 = nn.Linear(2,3)

nn.init.constant_(l3.weight[0,:], 1.0)
nn.init.constant_(l3.weight[1,0:2], 2.0)
nn.init.constant_(l3.weight[2,:], 3.0)
nn.init.constant_(l3.bias, 2.0)

l3.weight, l3.bias

(Parameter containing:
 tensor([[1., 1.],
         [2., 2.],
         [3., 3.]], requires_grad=True),
 Parameter containing:
 tensor([2., 2., 2.], requires_grad=True))

In [29]:
y3 = l3(x2)
y3.shape, y3.data

(torch.Size([4, 3]),
 tensor([[2., 2., 2.],
         [3., 4., 5.],
         [3., 4., 5.],
         [4., 6., 8.]]))

### 모델 정의

In [30]:
class Net(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.l1 = nn.Linear(n_input, n_output)

    def forward(self, x):
        x1 = self.l1(x)
        return x1

In [36]:
inputs = torch.ones(100,1)

n_input = 1
n_output = 1
net = Net(n_input, n_output)

outputs = net(inputs)
outputs.shape

torch.Size([100, 1])

In [32]:
criterion = nn.MSELoss()

loss = criterion(outputs, labels) / 2.

loss.backward()